# 导入库

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import os

# 读取原始数据

In [5]:
image_size=150

label_name=pd.read_csv('./train/train.csv')
label_name=np.array(label_name)
image_name=label_name[:,0]
labels=label_name[:,1]
#读取训练数据
images=[]
images_test=[]
for i in image_name:
    image=cv2.imread('./train/data/'+i)
    if image is None:
        image=img
    image=cv2.resize(image,(image_size,image_size))
    images.append(image)
    img = image
#读取测试数据
test_imgs = os.listdir('./test/')
for i in test_imgs:
    image=cv2.imread('./test/'+i)
    if image is None:
        image = img
    img = image
    image =cv2.resize(image,(image_size,image_size))
    images_test.append(image)

images = np.array(images)
labels = np.array(labels)
images_test = np.array(images_test)

labels_conv=np.zeros([labels.shape[0],5])
for i in range(labels_conv.shape[0]):
    labels_conv[i,labels[i]-1]=1
np.save('images.npy',images)
np.save('labels.npy',labels_conv)
np.save('images_test.npy',images_test)

# 导入vgg16网络，提取特征

In [2]:
from keras.applications.vgg16 import VGG16

model = VGG16(weights='imagenet', include_top=False)#第一次运行要下载一个 模型 


images=np.load('./images.npy')
images_test=np.load('./images_test.npy')
images_deel = model.predict(images)
images_test_deel = model.predict(images_test)
np.save('images_deel.npy',images_deel)
np.save('images_test_deel.npy',images_test_deel)

from sklearn.cross_validation import train_test_split
import random
import numpy as np
train_data = np.load('images_deel.npy')
label_data = np.load('./labels.npy')
#train_data = images_deel
#label_data = labele_conv
X_train, X_valid, y_train, y_valid = train_test_split(train_data, label_data, test_size=0.3, random_state=random.randint(0,100))

C:\Users\jerry\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


A local file was found, but it seems to be incomplete or outdated because the auto file hash does not match the original value of 6d6bbae143d832006294945121d1f1fc so we will re-download the data.
 9388032/58889256 [===>..........................] - ETA: 23:56

KeyboardInterrupt: 

# 建立模型 训练模型

In [5]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import SGD
from keras.utils import np_utils
from keras.models import load_model
from keras import backend as K
# the features were saved in order, so recreating the labels is easy

#validation_data = np.load('bottleneck_features_validation.npy')

model1 = Sequential()
model1.add(Flatten(input_shape=train_data.shape[1:]))
model1.add(Dense(256, activation='relu'))
model1.add(Dropout(0.5))
model1.add(Dense(5, activation='softmax'))#5类

model1.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model1.fit(X_train, y_train,
          epochs=10, batch_size=32,#循环次数10，可调
          validation_data=(X_valid, y_valid),
          shuffle=True)
model1.save_weights('bottleneck_fc_model.h5')

Train on 3197 samples, validate on 1371 samples
Epoch 1/10
3197/3197 [==============================] - 1s 442us/step - loss: 1.4110 - acc: 0.8647 - val_loss: 0.9428 - val_acc: 0.8985
Epoch 2/10
3197/3197 [==============================] - 1s 335us/step - loss: 0.7266 - acc: 0.8988 - val_loss: 0.5217 - val_acc: 0.8977
Epoch 3/10
3197/3197 [==============================] - 1s 314us/step - loss: 0.4593 - acc: 0.9136 - val_loss: 0.4734 - val_acc: 0.9053
Epoch 4/10
3197/3197 [==============================] - 1s 316us/step - loss: 0.3472 - acc: 0.9256 - val_loss: 0.4509 - val_acc: 0.9151
Epoch 5/10
3197/3197 [==============================] - 1s 314us/step - loss: 0.2979 - acc: 0.9394 - val_loss: 0.5418 - val_acc: 0.9104
Epoch 6/10
3197/3197 [==============================] - 1s 337us/step - loss: 0.2925 - acc: 0.9409 - val_loss: 0.5505 - val_acc: 0.9030
Epoch 7/10
3197/3197 [==============================] - 1s 327us/step - loss: 0.2552 - acc: 0.9498 - val_loss: 0.5824 - val_acc: 0.8998


# 预测结果，生成文件

In [7]:
#images_test_deel = np.load('images_test_deel.npy')
out = model1.predict(images_test_deel)

import os
out[1:10]
imgs = os.listdir("./test")
result1 = []
result2 = []
for i in range(out.shape[0]):
    for j in range(out.shape[1]):
        if out[i,j]==max(out[i]):
            result1.append(j+1)
            result2.append(imgs[i])
import pandas as pd
sub1 = pd.DataFrame({"filename":result2, "type":result1})
#sub1 = pd.DataFrame(result1)
sub1.to_csv('submission.csv', index=False)
sub1

,filename,type
0,996fdc7317bc49cb999926843a186d9a.jpg,4
1,0fab757cc07044479f68835df47b653b.jpg,3
2,78aec81a32414641956013942d931b21.jpg,4
3,4403f866403a42458ddcc3964edb1ff8.jpeg,2
4,503c079fa6b14781bce6dcc4083ee93f.jpg,2
5,9545885a9fe5425c890254f163b96030.jpg,4
6,699fc0323fbf476e9af3b4fb6766f273.jpg,4
7,b5eddd4db07a4c8da8923887558e55d0.jpg,3
8,93f78e9bee34401c9f50ddabb8e6c8f1.jpg,4
9,ba5e05c2136249789d48a54b325a2293.jpg,4
